# Wikipedia - Aspekte und Entitäten

Autor: Christian Koss - ckoss@uni-bremen.de

In [1]:
# Verwendete externe Packages
import pandas as pd
import sqlite3
import re
from collections import Counter
from urllib.request import urlopen

In [2]:
badtitle=[   'Diskussion:',
             'Benutzer Diskussion:',
             'Benutzer:',
             'Kategorie:',
             'Datei:',
             'Vorlage:',
             'Wikipedia:',
             'Portal:',
             'Wikipedia Diskussion:',
             'Kategorie Diskussion:',
             'Portal Diskussion:',
             'Vorlage Diskussion:',
             'MediaWiki:',
             'Datei Diskussion:',
             'Hilfe:',
             'Hilfe Diskussion:',
             'Modul:',
             'Modul Diskussion:',
             'ISO 3166-2:',
             'MediaWiki Diskussion:']

In [3]:
wikipedia_sql = 'data/wikipedia/wikipedia.db'

# Qualitative Auswahl durch vorherige Matching-Tests
kategorien=['Film','Fernsehsendung','Comic','Anime-Fernsehserie','Anime-Filme','Oper']

# Lade Wiki-Titel
conn=sqlite3.connect(wikipedia_sql)
cur = conn.cursor()
cur.execute('SELECT ID, title, infobox, text FROM wikipages WHERE infobox!="None"')

feature_dic={}
count=0
tags=[]
for ID, title, infobox, text in cur:
    if title.split(':')[0]+':' not in badtitle and infobox in kategorien:
        infobox=re.search('\{\{Infobox[^}]+\}\}',text, re.DOTALL).group(0)
        tags+=[tag.split('=')[0].replace('\n','').replace('|','').strip().lower() for tag in re.findall('\|[^|{}]+', infobox) if ' = ' in tag]
        count+=1        
conn.close()  

for tag in set(tags):
    feature_dic[tag]=None

db_length=count
print(count, ' Artikel extrahiert')         
print(len(tags), ' Entitäten gefunden')     
print(len(feature_dic), ' einzigartige Entitäten') 

43787  Artikel extrahiert
579398  Entitäten gefunden
374  einzigartige Entitäten


In [4]:
c=Counter(tags)
print(c)  

Counter({'musik': 31762, 'fsk': 30354, 'kamera': 28598, 'schnitt': 27197, 'ot': 24430, 'pl': 23199, 'os': 22987, 'len': 22199, 'dt': 19471, 'pro': 19122, 'originaltitel': 17952, 'produktionsland': 17135, 'originalsprache': 16932, 'länge': 16857, 'reg': 16252, 'pj': 16215, 'erscheinungsjahr': 16183, 'drb': 16026, 'deutscher titel': 14981, 'ds': 14747, 'bild': 14080, 'produzent': 12998, 'regie': 12635, 'drehbuch': 12327, 'jmk': 9541, 'besetzung': 9228, 'ej': 6590, 'genre': 5875, 'syn': 5684, 'eas': 4929, 'idee': 4759, 'sen': 4732, 'ea': 4618, 'st': 4104, 'synchronisation': 4001, 'produktionsunternehmen': 3751, 'id': 3327, 'sende': 3067, 'easde': 3058, 'song': 2462, 'chronologie': 1426, 'moderation': 1248, 'mus': 1247, 'lib': 1241, 'ua': 1233, 'uaort': 1220, 'el': 1163, 'erstausstrahlungsjahre': 1161, 't': 1123, 'ortzeit': 1101, 'regelmaessigkeit': 1094, 'dauer': 1047, 'form': 1035, 'sender_de': 1024, 'erstausstrahlung_de': 990, 'nds': 946, 'staffeln': 940, 'episoden': 938, 'sender': 938,

In [5]:
# siehe: https://de.wikipedia.org/wiki/Vorlage:Infobox_Film
with urlopen("https://de.wikipedia.org/wiki/Vorlage:Infobox_Film") as conn:
    conn=str(conn.read().decode('utf-8')).split('<a href="#Parameter-Details">')[1].split('<a href="#H\xc3\xa4ufig_gestellte_Fragen_(FAQ)">')[0]
    tags1=re.findall('<dt>([^<]+\([^<]+\))<\/dt>',conn)
    
tags1=[(tag.split('(')[0].strip() , tag.split('(')[1].split(')')[0].split(',')[0].strip()) for tag in tags1]+[('Erscheinungsjahr','PJ')]
# jmk = Jugendmedienkommission (Altersfreigabe vergleichbar mit fsk)


# siehe: https://de.wikipedia.org/wiki/Vorlage:Infobox_Fernsehsendung
with urlopen("https://de.wikipedia.org/wiki/Vorlage:Infobox_Fernsehsendung") as conn:
    conn=str(conn.read().decode('utf-8')).split('<table')[1].split('</table>')[0]
    tags2=re.findall('>([^<]+)<\/td><td><code[^<]+<\/code>(.*?)<\/td>',conn.replace('\n','').replace('<br />',''))
    
tags=set(tags1+tags2)
schablone={full.lower():'' for full,abbr in tags}
schablone['schauspieler']=''
schablone['figur']=''
replace_tag={abbr.lower():full.lower() for full,abbr in tags if abbr!=''}
print(replace_tag)

{'eas': 'erstausstrahlung', 'drb': 'drehbuch', 'ea': 'episoden', 'schnitt': 'schnitt', 'pro': 'produzent', 'sen': 'sender', 'syn': 'synchronisation', 'evöde': 'erstveröffentlichung de', 'chr': 'chronologie', 'db': 'drehbuch', 'musik': 'musik', 'el': 'episodenliste', 'kamera': 'kamera', 'evö': 'erstveröffentlichung', 'regelmaessigkeit': 'regelmäßigkeit', 'sende': 'sender de', 'reg': 'regie', 'len': 'länge', 'ot': 'originaltitel', 'produktionsunternehmen': 'produktionsunternehmen', 'song': 'titellied', 'pj': 'erscheinungsjahr', 'pl': 'produktionsland', 'easde': 'erstausstrahlung de', 'dt': 'deutscher titel', 'st': 'staffeln', 'genre': 'genre', 'moderation': 'moderation', 'os': 'originalsprache', 'nds': 'nebendarsteller', 'id': 'bild', 'rg': 'regie', 'idee': 'idee', 'ds': 'besetzung', 'ej': 'erscheinungsjahr'}


In [6]:
#fix String
def char_replace(String):
    
    subit="ңүאבדהוזחטיךכלםמןנסעףפצקרשתآابتجحخدرزسشصفقلمنهويیंएकगचजञटतदनपभमयरलवशसहािीुूृेो्কঘঢতমরােਅਘਦਨਹਾੇੋ੍ੜੰகசடதநமயரலவிீுெ்ಕಣಧಮರವಿೀ್กขงชญตนบมยรลวสอะัาิีึุ้་ཁགངཅཆཐབའལསཧིེུྣྲაბდევზიკლმნორსუღḥṣạầộờợủứừữ‌‍‎‒†•″‹›⅓→−≈★♪、。々《》〜あいうえおかがきぎくぐけげこごさざしじすずせそただちっつてでとどなにぬのはばひびふぶぷべぼまみむめもゃやゅゆよらりるれろわをんゝアィイウェエォオカガキギクグケコゴサザシジスズセソゾタダチッツテデトドナニヌネハバパヒビピフブプヘベペボポマミムメモャヤュユョヨラリルレロ七万三下不世个丸主之乱了事京人仁代仲件任休伝传伽住佐体作你佬使侠保俠候倩停健偵偷傑元兄先光兔党冒军凌别刺劃劇助勇動勝北十单南印原取古叶号各名君味命和唐問喜喰嘉囚四団国图國園在地城場墓壁士夏外夢大天太奇契女好妄妈妓妹姉姬娘婚子学孫宇守宙定客室宫宴家宿察寶対寿小少屋山岁岬島市帝師席帰年幽庭开归影往後忍快怨怪恐息悟您悪想愛感戀战戦才招探放教新方旅族无日昏春時晴曜曹會月有未本杂村来杰東极根権樽橋機武歲殺毛水決洋流海滸漆激烙無爆爸物狂狄狼猿獄獣玻球理璃生產甲男界畑疆発的皇盗監督瞳破社祇祖祝神福种穹空突竜竹節篇約素結絲絵緊縛绣美義者能脳腕膵臓色花若英茶草菊華落葉號裂装補西見角計記詭誕語説課諜警護豆貝貸赤起超車軌軍軒輪车通連遊道郎部都醜金鉄錬鑓铁門間闘问闹阪降限陸険陽隠雄雅雙雞雨電霸青静頃顶風飛餃駅髪魂魔鳥鹿麗黄黑黒龍간것경공구극금기꿈나날남내냥녀놈는다달답도동라럴레를름리림마맨며몽뫼미바발법보복브비빈쁜상생섬성소수숨스시실씨아악았에역열왕우워은의이인자재절정좋주줌집츄친커콤타태튜티팔피한휘"
    Sub=[('A','ÀÁÂÃÅ'),('Ae','Æ'),('E','ÈÉÊ'),('I','ÌÍÎÏİ'),('O','ÓØŌ'),('U','ÚÛÚ'),('d','ďđ'),
             ('a','àáâãåāăąả'),('ae','æ'),('c','çćč'),('C','ÇĆÇČ'),('e','èéêëếēėęě'),('i','ìíîïịīı'),('n','ñńņň'),
             ('o','òóôøọỗō'),('ö','őõ'),('Ö','Ő'),('t','țţť'),('u','ùúûūů'),('u','ùúû'),('Oe','Œ'),('oe','œ'),
             ('S','ŠŚŞȘ'),('s','šșşś'),('Y','Ÿ'),('Z','ŽŻ'),('z','žż'),('ß','β'),('...','…'),('D','ĎĐ'),('d','ð'),
             ("""'""",'´′’`'),('l','ĺļľł'),('L','ĽŁ'),('R','Ř'),('r','ř'),('k','ķ'),('K','Ķ'),
             ('y','ýÿÿý'),('y','ýÿ'),('g','ğ'),('G','Ģ'),('p','Þþ'),('','中华慧文昆林王白')]
    for new,old in Sub:
        String=re.sub('['+old+']',new, String)

    Replace=[('&amp;',' und '),('&lt;',''), ('&gt;',''),('&',' und '),
             ('\xad',''),('\x84',''),('\x9c','Ü'),('\x9f','ß'),('\xc3\xb6','ö'),('\xc3\xbc','ü'),('\xc3\xa4','ä'),
             ('A¤','ä'),('A¶','ö'),('A¼','ü'),('~',''),('=',' '),('_',' '),
             ('*',' '),('+',' '),('¬',''),('²',''),('³',''),('au§er','außer'),('Spa§','Spaß')]

    
    for old,new in Replace:
        String=String.replace(old,new)
        
    String=re.sub('[#%&@^_¡£§©«®°¶·»¼½–—‘‚“”„€]','', String)
    String=re.sub('['+subit+']','', String)
    
    #Standard
    String=re.sub('[.!?,;]', ' . ', String).replace('-',' ')
    String=re.sub('[\\\/\{\[\]}\(\)\"\'\:]', ' ', String)
    String=re.sub('\s+',' ', String).strip()
    return String

# extrahiere Infobox
def re_infobox(text):
    nested=0
    match=False
    search=''
    for l in text:
        if '{' == l:
            nested=nested+1
            match=True
            
        if '}' == l:
            nested=nested-1
            
        if nested!=0 and match==True:
            search+=l
         
        if nested==0 and match==True and '{{infobox' in search.lower():
            break;
        else:            
            if nested==0 and match==True:
                search=''
    search=search[2:-2].replace('<br />','|').replace('\n',' ') 
        
        #cleaning
    return search 

# extrahiere Features aus der Infobox
def re_features(text):
    nested=0
    feature=[]
    search=''
    for l in text:
        if '[' == l or '{' == l or '(' == l:
            nested=nested+1
            
        if ']' == l or '}' == l or ')' == l:
            nested=nested-1
            
        search+=l
        
        if l=='|' and nested==0:
            feature.append(search)
            search=''              
    features=[f.split('=',1) for f in feature if '=' in f]
    features={k.strip():[re.sub('<.*>','',f.strip()) for f in v.replace('[','').replace(']','').replace('\n|','').split('\n*') if f.strip()!=''] for k,v in features}
    
    #feature selection
    actors=[]
    roles=[]
    
    try:
        features['Besetzung']=features['Besetzung'][0]
    except:
        features['Besetzung']=''

    for k in ['ds','DS','darsteller','Darsteller','nds','NDS','nebendarsteller','Nebendarsteller']:
        try:
            features['Besetzung']+=';'+features[k][0]
        except:
            pass

    try:
        temp=[t.split(':') for t in features['Besetzung'].replace(';','*').replace('#','').split('*') if t!='']
        for actor, role in temp:
            actors+=actor.replace('| ','|').split('|')
            actors=[a.strip() for a in actors]
            roles+=role.replace(')','').replace('(',',').strip().split(',')
            roles=[r.strip() for r in roles]
        #del features['Besetzung']
        features['Schauspieler']=actors
        features['Figur']=roles
    except:
        pass
    
    return {tag:'|'.join(features[tag]).strip('|').strip() for tag in list(features)}

In [7]:
wikipedia_sql = 'data/wikipedia/wikipedia.db'
kategorien=['Film','Fernsehsendung','Comic','Anime-Fernsehserie','Anime-Filme','Oper']

# Lade Wiki-Titel
conn=sqlite3.connect(wikipedia_sql)
cur = conn.cursor()

cur.execute('SELECT ID, title, infobox, text FROM wikipages WHERE infobox!="None"')

feature_dic={}
count=0
tags=[]
data=[]
for ID, title, infobox, text in cur:
    if title.split(':')[0]+':' not in badtitle and infobox in kategorien:
        infobox_text=re_infobox(text)
        feature_dic=re_features(infobox_text)
        
        temp_schablone=schablone.copy()
        temp_schablone['schauspieler']=''
        temp_schablone['figur']=''
        for k,v in feature_dic.items():
            k=k.lower()
            try:
                k=replace_tag[k]
            except:
                pass
            try:
                temp_schablone[k]
                temp_schablone[k]=v
            except:
                pass  
        # lösche unnötige Daten
        for tag in ['bild','bildhöhe','besetzung','ds','DS','darsteller','Darsteller','nds','NDS','nebendarsteller','Nebendarsteller']:
            temp_schablone.pop(tag, None)

        line=[ID,infobox,title]+[v for k,v in temp_schablone.items()]
        data.append(line)
        count+=1        
        if count%10000==0 or count==db_length:
            print(count,'/',db_length, ' Daten extrahiert')
conn.close()  

10000 / 43787  Daten extrahiert
20000 / 43787  Daten extrahiert
30000 / 43787  Daten extrahiert
40000 / 43787  Daten extrahiert
43787 / 43787  Daten extrahiert


In [8]:
df=pd.DataFrame(data, columns=['ID','infobox','title']+list(temp_schablone))
df[:5]

,ID,infobox,title,franchise,originalsprache,produzent,episodenliste,sender de,schauspieler,deutscher titel,...,originaltitel,erstausstrahlung de,episoden,genre,idee,erstveröffentlichung,regelmäßigkeit,produktionsland,figur,kamera
0,177121967,Film,Welt am Draht,,Deutsche Sprache|Deutsch,Peter Märthesheimer,,,Klaus Löwitsch|Barbara Valentin|Mascha Rabben|...,,...,Welt am Draht,,,,,,,Deutschland|Bundesrepublik Deutschland,Fred Stiller|Gloria Fromm|Eva Vollmer|Herbert ...,Michael Ballhaus
1,167493890,Film,Die Familie mit dem umgekehrten Düsenantrieb,,Japanische Sprache|Japanisch,Banmei Takahashi,,,,Die Familie mit dem umgekehrten Düsenantrieb,...,Gyakufunsha Kazoku,,,,,,,Japan,,Masaki Tamura
2,179017630,Oper,Der fliegende Holländer,,Deutsch,,,,,,...,Der fliegende Holländer,,,,,,,,,
3,177449155,Fernsehsendung,Die Pyramide (Fernsehsendung),,Deutsche Sprache|Deutsch,,,,,,...,Die Pyramide,,165,Quizsendung,,,unregelmäßig,Deutschland|Bundesrepublik Deutschland,,
4,179802296,Film,Pulp Fiction,,Englische Sprache|Englisch,Lawrence Bender,,,John Travolta|Samuel L. Jackson|Uma Thurman|Br...,Pulp Fiction,...,Pulp Fiction,,,,,,,Vereinigte Staaten,Vincent Vega|Jules Winnfield|Mia Wallace|Butch...,Andrzej Sekuła


In [9]:
# Pandas -> SQL
db='infobox'
wikipedia_sql = 'data/wikipedia/infobox.db'

conn = sqlite3.connect(wikipedia_sql)
cur = conn.cursor()                                 

wildcards = ','.join(['?'] * len(df.columns))              

cur.execute("drop table if exists %s" % db)

col_str = '"' + '","'.join(df.columns) + '"'
cur.execute("create table %s (%s)" % (db, col_str))

cur.executemany("insert into %s values(%s)" % (db, wildcards), [tuple(x) for x in df.values])

conn.commit()
conn.close()

In [10]:
df.columns[3:]

Index(['franchise', 'originalsprache', 'produzent', 'episodenliste',
       'sender de', 'schauspieler', 'deutscher titel', 'chronologie',
       'produktionsunternehmen', 'titellied', 'erscheinungsjahr',
       'synchronisation', 'erstveröffentlichung de', 'erscheinungsjahre',
       'länge', 'schnitt', 'sender', 'erstausstrahlung', 'staffeln', 'musik',
       'regie', 'drehbuch', 'moderation', 'originaltitel',
       'erstausstrahlung de', 'episoden', 'genre', 'idee',
       'erstveröffentlichung', 'regelmäßigkeit', 'produktionsland', 'figur',
       'kamera'],
      dtype='object')

In [11]:
#Zeichenliste / Cleaning

chars=''
for c in df.columns[3:]:
    temp=[]
    for actors in df[c]:
        if actors!='':
            actors=re.sub('\([^\)]+\)','',actors)
            actors=re.sub('\{\{[^\})]+\}\}','',actors).replace('/','|').replace('%C3%B8','ø')
            actors=char_replace(actors)
            actors=[actor.strip() for actor in sorted(set(actors.split('|'))) if actor.strip()!='']
            actors='|'.join(actors)
        temp.append(actors)
    print('cleaning:', c)
    df[c]=temp
    chars+=''.join(sorted(set(''.join(df[c]))))
chars=''.join(sorted(set(''.join(chars))))
print(chars)

cleaning: franchise
cleaning: originalsprache
cleaning: produzent
cleaning: episodenliste
cleaning: sender de
cleaning: schauspieler
cleaning: deutscher titel
cleaning: chronologie
cleaning: produktionsunternehmen
cleaning: titellied
cleaning: erscheinungsjahr
cleaning: synchronisation
cleaning: erstveröffentlichung de
cleaning: erscheinungsjahre
cleaning: länge
cleaning: schnitt
cleaning: sender
cleaning: erstausstrahlung
cleaning: staffeln
cleaning: musik
cleaning: regie
cleaning: drehbuch
cleaning: moderation
cleaning: originaltitel
cleaning: erstausstrahlung de
cleaning: episoden
cleaning: genre
cleaning: idee
cleaning: erstveröffentlichung
cleaning: regelmäßigkeit
cleaning: produktionsland
cleaning: figur
cleaning: kamera
 $.0123456789<>ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz|¢ª¿ÄÐÔÕÖ×ÜßäöüĕŏŤŭűźưǎǐǒǔǜǧȚəʻʿ́ͤΆΈΌΑΓΔΕΗΙΚΛΜΞΟΠΣΤΨΩάέήίαγδεζηθικλμνοπρςστυφχωόЁЋАБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЭЯабвгдежзийклмнопрстуфхцчшщъыьэюяёіјワンヴ・ー一！（，０２～


In [12]:
# Pandas -> SQL
db='infobox'
wikipedia_sql = 'data/wikipedia/infobox.db'

conn = sqlite3.connect(wikipedia_sql)
cur = conn.cursor()                                 

wildcards = ','.join(['?'] * len(df.columns))              

cur.execute("drop table if exists %s" % db)

col_str = '"' + '","'.join(df.columns) + '"'
cur.execute("create table %s (%s)" % (db, col_str))

cur.executemany("insert into %s values(%s)" % (db, wildcards), [tuple(x) for x in df.values])

conn.commit()
conn.close()

In [13]:
cleandf=df.copy()
df[:5]

,ID,infobox,title,franchise,originalsprache,produzent,episodenliste,sender de,schauspieler,deutscher titel,...,originaltitel,erstausstrahlung de,episoden,genre,idee,erstveröffentlichung,regelmäßigkeit,produktionsland,figur,kamera
0,177121967,Film,Welt am Draht,,Deutsch|Deutsche Sprache,Peter Märthesheimer,,,Adrian Hoven|Barbara Valentin|Bruce Low|Christ...,,...,Welt am Draht,,,,,,,Bundesrepublik Deutschland|Deutschland,Assistentin im Transferierungsraum|Betriebsarz...,Michael Ballhaus
1,167493890,Film,Die Familie mit dem umgekehrten Düsenantrieb,,Japanisch|Japanische Sprache,Banmei Takahashi,,,,Die Familie mit dem umgekehrten Düsenantrieb,...,Gyakufunsha Kazoku,,,,,,,Japan,,Masaki Tamura
2,179017630,Oper,Der fliegende Holländer,,Deutsch,,,,,,...,Der fliegende Holländer,,,,,,,,,
3,177449155,Fernsehsendung,Die Pyramide (Fernsehsendung),,Deutsch|Deutsche Sprache,,,,,,...,Die Pyramide,,165,Quizsendung,,,unregelmäßig,Bundesrepublik Deutschland|Deutschland,,
4,179802296,Film,Pulp Fiction,,Englisch|Englische Sprache,Lawrence Bender,,,Alexis Arquette|Amanda Plummer|Angela Jones|Br...,Pulp Fiction,...,Pulp Fiction,,,,,,,Vereinigte Staaten,Brett|Buddy Holly|Butch Coolidge|Cpt . Koons|E...,Andrzej Sekula


In [14]:
df.columns=[col.replace('title','title_0').replace('deutscher titel','title_1').replace('originaltitel','title_2') for col in df.columns]

In [15]:
def repit(String):
    for rep in re.findall('[a-zöüä][A-ZÄÖÜ]',String):
        String=String.replace(rep,rep[0]+'|'+rep[1])
    return String.replace('.','').replace(',','')

relevant=['figur', 'idee', 'musik','kamera','title_0', 'title_1','title_2', 'schnitt','genre','schauspieler','produzent','drehbuch','synchronisation']

ner=[]
token={}
pos={}
entity={}
ID={}

count=len(df)
i=0
for row in df.iterrows():
    for tag in relevant:
        tokens=repit(row[1][tag]).split('|')
        for tok in tokens:
            parts=tok.split(' ')
            for part in parts:
                if part!='':
                    token[(row[1]['ID'], part)]=None
                    for dic,target in [(pos,tag),(entity,tok),(ID,row[1]['ID'])]:
                        try:
                            if target!='':
                                dic[(row[1]['ID'], part)]+='|'+target.split('_')[0].strip()
                        except:
                            dic[(row[1]['ID'], part)]=target
    i+=1
    if i%5000==0 or i==count:
        print(i,'/',count)

5000 / 43787
10000 / 43787
15000 / 43787
20000 / 43787
25000 / 43787
30000 / 43787
35000 / 43787
40000 / 43787
43787 / 43787


In [16]:
def set_it(String):
    String=set(String.split('|'))
    String=re.sub('\([^\)]\)','','|'.join(String))
    return String

def is_unigue(String,entity):
    if '|' not in String.strip('|') or '|' not in entity.strip('|'):
        return 'True'
    else:
        return 'False'

ner_df=pd.DataFrame([(key[0]+'|'+key[1],key[1], set_it(pos[key]),is_unigue(set_it(pos[key]),set_it(entity[key])),set_it(entity[key]), set_it(ID[key])) for key in token.keys()],columns=['key','token','pos','unique_entity','entity','ID'])

In [17]:
ner_df['entity']='entity'
ner_df[:10]

,key,token,pos,unique_entity,entity,ID
0,145315562|Dailland,Dailland,kamera,True,entity,145315562
1,178014195|Alan,Alan,musik,True,entity,178014195
2,166864366|Maschke,Maschke,produzent,True,entity,166864366
3,180036564|Jonathan,Jonathan,schauspieler,True,entity,180036564
4,151710626|Franz,Franz,produzent,True,entity,151710626
5,179842521|des,des,title|title_0,True,entity,179842521
6,180160366|Mario,Mario,schauspieler,True,entity,180160366
7,180012751|Elle,Elle,schauspieler,True,entity,180012751
8,163192958|Fernando,Fernando,schauspieler,True,entity,163192958
9,179764645|s,s,title|title_1,True,entity,179764645


In [18]:
db='ner' 
df=ner_df
conn = sqlite3.connect('data/data.db')
cur = conn.cursor()                                 

wildcards = ','.join(['?'] * len(df.columns))              

cur.execute("drop table if exists %s" % db)

col_str = '"' + '","'.join(df.columns) + '"'
cur.execute("create table %s (%s)" % (db, col_str))

cur.executemany("insert into %s values(%s)" % (db, wildcards), [tuple(x) for x in df.values])

conn.commit()
conn.close()

In [19]:
df=cleandf.copy()
df.columns=[col.replace('title','title_0').replace('deutscher titel','title_1').replace('originaltitel','title_2') for col in df.columns]
relevant=['figur', 'idee', 'musik','kamera','title_0', 'title_1','title_2', 'schnitt','genre','schauspieler','produzent','drehbuch','synchronisation']

ner=[]
token={}
pos={}
entity={}
ID={}

count=len(df)
i=0
for row in df.iterrows():
    for tag in relevant:
        tokens=repit(row[1][tag]).split('|')
        for tok in tokens:
            parts=tok.split(' ')
            for part in parts:
                if part!='':
                    token[part]=None
                    for dic,target in [(pos,tag),(entity,tok),(ID,row[1]['ID'])]:
                        try:
                            if target!='':
                                dic[part]+='|'+target.split('_')[0].strip()
                        except:
                            dic[part]=target
    i+=1
    if i%5000==0 or i==count:
        print(i,'/',count)

5000 / 43787
10000 / 43787
15000 / 43787
20000 / 43787
25000 / 43787
30000 / 43787
35000 / 43787
40000 / 43787
43787 / 43787


In [20]:
ner2_df=pd.DataFrame([(key, set_it(pos[key]),is_unigue(set_it(pos[key]),set_it(entity[key])),set_it(entity[key]), set_it(ID[key])) for key in token.keys()],columns=['token','pos','unique','entity','ID'])
ner2_df['entity']='entity'
ner2_df[:10]

,token,pos,unique,entity,ID
0,Batisti,figur,True,entity,179557703
1,Colonnesi,figur,True,entity,158920814
2,Laternenanzünder,figur,True,entity,170440869
3,Make-up,title_0,True,entity,174381190
4,Jaffray,figur|musik,False,entity,178224676|142271056
5,Gamle,title_2,True,entity,180029997
6,Passi,kamera|title_2,False,entity,164562131|172142711|179842521
7,Wade,title|produzent|schnitt|schauspieler|musik|kam...,False,entity,159895374|164948904|176796670|179559249|176823...
8,Sandbaf,schnitt,True,entity,175732902
9,Ouassini,schauspieler,True,entity,176931582


In [21]:
db='ner2' 
df=ner2_df
conn = sqlite3.connect('data/data.db')
cur = conn.cursor()                                 

wildcards = ','.join(['?'] * len(df.columns))              

cur.execute("drop table if exists %s" % db)

col_str = '"' + '","'.join(df.columns) + '"'
cur.execute("create table %s (%s)" % (db, col_str))

cur.executemany("insert into %s values(%s)" % (db, wildcards), [tuple(x) for x in df.values])

conn.commit()
conn.close()